In [43]:
execfile('../spark-scripts/split.py')
execfile('../spark-scripts/utils.py')
execfile('../spark-scripts/eval.py')
execfile('../spark-scripts/SAGHFunctions.py')
execfile('../spark-scripts/SAGHMain.py')

In [44]:
import json
import copy

conf = {}

conf['split'] = {}
conf['split']['reclistSize'] = 100
conf['split']['callParams'] = {}
conf['split']['excludeAlreadyListenedTest'] = True
conf['split']['name'] = '0_1SPLIT_1413851857_sessionsession_1413851857'
conf['split']['minEventsPerUser'] = 5
conf['split']['inputData'] = 's3n://contentwise-research-poli/0_1SPLIT_1413851857_sessionsession_1413851857'
#conf['split']['inputData'] = 's3n://contentwise-research-poli/30musicdataset/newFormat/relations/sessions.idomaar'
conf['split']['bucketName'] = 'contentwise-research-poli'
conf['split']['percUsTr'] = 0.05
conf['split']['ts'] = int(0.75 * (1421745857 - 1390209860) + 1390209860) - 10000
conf['split']['minEventPerSession'] = 5
conf['split']['onlineTrainingLength'] = 5
conf['split']['GTlength'] = 5
conf['split']['minEventPerSessionTraining'] = 10
conf['split']['minEventPerSessionTest'] = 11
conf['split']['mode'] = 'session'
conf['split']['forceSplitCreation'] = False

conf['evaluation'] = {}
conf['evaluation']['metric'] = {}
conf['evaluation']['metric']['type'] = 'recall'
conf['evaluation']['metric']['prop'] = {}
conf['evaluation']['metric']['prop']['N'] = [1,2,5,10,15,20,25,50,100]
conf['evaluation']['name'] = 'recall@N'

conf['general'] = {}
conf['general']['clientname'] = 'split2.split'
conf['general']['bucketName'] = 'contentwise-research-poli'
conf['general']['tracksPath'] = '30Mdataset/entities/tracks.idomaar.gz'

conf['algo'] = {}
conf['algo']['name'] = 'SAGH'
conf['algo']['props'] = {}
# ***** EXAMPLE OF CONFIGURATION *****#
conf['algo']['props']['numGH'] = 10
conf['algo']['props']['skipTh'] = 0
# ****** END EXAMPLE ****************#

In [7]:
artistLookupRDD = loadArtistLookup(conf)
train,test = loadDataset(conf)
artistLookupRDD.cache()
train.cache()
test.cache()

In [45]:
from os import path
basePath = path.join("s3n://", conf['general']['bucketName'], conf['general']['clientname'])
splitPath = path.join(basePath, conf['split']['name'])

numGHList = [10, 50, 100, 250]
skipThList = [0, 0.05]

numGHList = [10]
skipThList = [0]


for th in skipThList:
    conf['algo']['props']['skipTh'] = th
    batchTrainingRDD = (train
                        .flatMap(lambda x: ext(json.loads(x))).filter(lambda x: x[1] > th)
                        .map(lambda x: (int(x[0]), int(x[2])))
                        .cache())
    recReqRDD = parseRequests(artistLookupRDD, test, th, conf).cache()
    
    for numGH in numGHList:
        recJsonRdd = generateRecommendationsSAGH(batchTrainingRDD, recReqRDD, artistLookupRDD, test, numGH, conf)
        saveRecommendations(conf, recJsonRdd, overwrite=True)
        computeMetrics(conf)


split/giroCompletoTestMultipleConfs_excludeTrue/Rec/SAGH_10#0 successfully deleted.
Recommendations successfully written to s3n://contentwise-research-poli/split/giroCompletoTestMultipleConfs_excludeTrue/Rec/SAGH_10#0/recommendations
recall@N successfully written to s3n://contentwise-research-poli/split/giroCompletoTestMultipleConfs_excludeTrue/Rec/SAGH_10#0/recall@N/metrics


In [53]:
    basePath = path.join("s3n://", conf['general']['bucketName'], conf['general']['clientname'])
    # basePath = "s3n://" + conf['general']['bucketName'] + "/"+conf['general']['clientname']+"/"
    splitPath = path.join(basePath, conf['split']['name'])
    # splitPath = basePath+conf['split']['name']+"/"
    GTpath = path.join(splitPath, "GT")
    # GTpath = splitPath+"GT"

    algo_conf = conf['algo']['name'] + '_' + \
        '#'.join([str(v) for k, v in conf['algo']['props'].iteritems() if not k.startswith('libFM')])
    confPath = path.join(splitPath, 'Rec', algo_conf)
    recPath = path.join(confPath, "recommendations")
    # recPath = splitPath+"/Rec/"+ conf['algo']['name']+"/recommendations/"

    gtRDD = sc.textFile(GTpath).map(lambda x: json.loads(x)).persist(StorageLevel.MEMORY_AND_DISK)
    recRDD = sc.textFile(recPath).map(lambda x: json.loads(x)).persist(StorageLevel.MEMORY_AND_DISK)

    recommendationRDD = recRDD \
        .flatMap(lambda x: ([(x['id'], (k['id'], k['rank'], x)) for k in x['linkedinfo']['response']]))

    groundTruthRDD = gtRDD \
        .flatMap(lambda x: ([(x['linkedinfo']['gt'][0]['id'], (k['id'], x)) for k in x['linkedinfo']['objects']]))
    
    hitRDDPart = recommendationRDD.join(groundTruthRDD).filter(lambda x: x[1][0][0] == x[1][1][0])
    hitRDD = hitRDDPart.map(lambda x: (x[0], x[1][0][1], 1.0))


In [64]:
groundTruthRDD.take(10)

[(4096001420015097,
  (281050,
   {u'id': u'136127',
    u'linkedinfo': {u'gt': [{u'id': 4096001420015097, u'type': u'request'}],
     u'objects': [{u'action': u'play',
       u'id': 281050,
       u'playratio': 1.0,
       u'playstart': 1410,
       u'playtime': 253,
       u'type': u'track'},
      {u'action': u'play',
       u'id': 3219635,
       u'playratio': 1.22,
       u'playstart': 1663,
       u'playtime': 244,
       u'type': u'track'},
      {u'action': u'play',
       u'id': 716858,
       u'playratio': 0.96,
       u'playstart': 1907,
       u'playtime': 238,
       u'type': u'track'},
      {u'action': u'play',
       u'id': 3068798,
       u'playratio': 1.16,
       u'playstart': 2398,
       u'playtime': 270,
       u'type': u'track'},
      {u'action': u'play',
       u'id': 2577438,
       u'playratio': 1.0,
       u'playstart': 2668,
       u'playtime': 213,
       u'type': u'track'}],
     u'subjects': [{u'id': 40960, u'type': u'user'}]},
    u'ts': u'1420015097',


In [57]:
gtRDD.take(1)

[{u'id': u'136127',
  u'linkedinfo': {u'gt': [{u'id': 4096001420015097, u'type': u'request'}],
   u'objects': [{u'action': u'play',
     u'id': 281050,
     u'playratio': 1.0,
     u'playstart': 1410,
     u'playtime': 253,
     u'type': u'track'},
    {u'action': u'play',
     u'id': 3219635,
     u'playratio': 1.22,
     u'playstart': 1663,
     u'playtime': 244,
     u'type': u'track'},
    {u'action': u'play',
     u'id': 716858,
     u'playratio': 0.96,
     u'playstart': 1907,
     u'playtime': 238,
     u'type': u'track'},
    {u'action': u'play',
     u'id': 3068798,
     u'playratio': 1.16,
     u'playstart': 2398,
     u'playtime': 270,
     u'type': u'track'},
    {u'action': u'play',
     u'id': 2577438,
     u'playratio': 1.0,
     u'playstart': 2668,
     u'playtime': 213,
     u'type': u'track'}],
   u'subjects': [{u'id': 40960, u'type': u'user'}]},
  u'ts': u'1420015097',
  u'type': u'event.session'}]

In [63]:
hitRDDPart.take(10)

[(781201419787782,
  ((2441445,
    0,
    {u'id': 781201419787782,
     u'linkedinfo': {u'objects': [{u'action': u'play',
        u'id': 2441448,
        u'playratio': 0.1,
        u'playstart': 0,
        u'playtime': 47,
        u'type': u'track'},
       {u'action': u'play',
        u'id': 758777,
        u'playratio': None,
        u'playstart': 47,
        u'playtime': 158,
        u'type': u'track'},
       {u'action': u'play',
        u'id': 2441482,
        u'playratio': 0.83,
        u'playstart': 205,
        u'playtime': 170,
        u'type': u'track'},
       {u'action': u'play',
        u'id': 2441486,
        u'playratio': 1.66,
        u'playstart': 375,
        u'playtime': 281,
        u'type': u'track'},
       {u'action': u'play',
        u'id': 2441467,
        u'playratio': 1.16,
        u'playstart': 656,
        u'playtime': 331,
        u'type': u'track'}],
      u'response': [{u'id': 2441445, u'rank': 0, u'type': u'track'},
       {u'id': 2441463, u'rank': 1, 

In [54]:
hitRDD.take(10)

[(781201419787782, 0, 1.0),
 (781201419787782, 3, 1.0),
 (2633201420044230, 3, 1.0),
 (3314101420238346, 3, 1.0),
 (90801417460086, 6, 1.0),
 (324901421154054, 3, 1.0),
 (324901421154054, 16, 1.0),
 (1412301421514286, 0, 1.0),
 (3266201415817782, 0, 1.0),
 (3266201415817782, 1, 1.0)]